# Analyse de la performance d’une stratégie GARP (Growth at a Reasonable Price)

**Master 2 Finance Internationale**

Objectif :
Analyser la performance d’une stratégie de conviction GARP, en décomposant l’alpha,
et en évaluant la persistance du couple rendement–risque face à des benchmarks mondiaux.


In [1]:
# =========================
# INSTALLATION DES PACKAGES
# =========================

! pip install pandas numpy yfinance matplotlib seaborn statsmodels openpyxl scikit-learn


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 73.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 86.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 74.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 74.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 86.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 67.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 56.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 67.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 79.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 52.8 MB/s  0:00:006m0:00:0100:01
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15635 sha256=a2feb

## 1. Préparation des données - Portefeuille GARP

Cette section vise à importer, nettoyer et harmoniser les données nécessaires à l’analyse empirique :
- Actions du portefeuille GARP (Yahoo Finance)
- Devise homogène : EUR
- Fréquence : mensuelle


In [2]:
# ======================================================
# 1. PREPARATION DES DONNEES – PORTEFEUILLE GARP
# ======================================================

# ----------------------
# 1.1 Librairies
# ----------------------
import pandas as pd
import numpy as np
import yfinance as yf
from pathlib import Path


# ----------------------
# 1.2 Univers d’investissement
# ----------------------
# Portefeuille GARP composé de 10 actions internationales
# La devise associée à chaque titre est précisée afin de
# permettre une conversion rigoureuse en euros

garp_stocks = {
    "ADYEN.AS": "EUR",
    "KRI.AT": "EUR",
    "AMZN": "USD",
    "META": "USD",
    "GOOGL": "USD",
    "CSU.TO": "CAD",
    "MELI": "USD",
    "DNP.WA": "PLN",
    "LLY": "USD",
    "KNSL": "USD"
}


# ----------------------
# 1.3 Période d’étude
# ----------------------
start_date = "2021-02-01"
end_date = "2026-01-31"


# ----------------------
# 1.4 Téléchargement des prix ajustés
# ----------------------
# Les prix ajustés (auto_adjust=True) incluent les dividendes
# et sont extraits depuis Yahoo Finance

prices = yf.download(
    tickers=list(garp_stocks.keys()),
    start=start_date,
    end=end_date,
    auto_adjust=True,
    progress=False
)["Close"]


# ----------------------
# 1.5 Passage en fréquence mensuelle
# ----------------------
# Les prix sont retenus en fin de mois afin d’être cohérents
# avec les benchmarks utilisés dans l’analyse ultérieure

prices_m = prices.resample("ME").last()


# ----------------------
# 1.6 Conversion des prix en euros
# ----------------------

# Tickers de change Yahoo Finance (devise locale -> EUR)
fx_tickers = {
    "USD": "USDEUR=X",
    "CAD": "CADEUR=X",
    "PLN": "PLNEUR=X"
}

# Téléchargement des taux de change
fx = yf.download(
    tickers=list(fx_tickers.values()),
    start=start_date,
    auto_adjust=True,
    progress=False
)["Close"]

# Passage en fréquence mensuelle
fx = fx.resample("ME").last()
fx.columns = fx_tickers.keys()

# Conversion des prix non libellés en EUR
prices_eur = prices_m.copy()

for ticker, currency in garp_stocks.items():
    if currency != "EUR":
        prices_eur[ticker] = prices_m[ticker] * fx[currency]


# ----------------------
# 1.7 Calcul des rendements logarithmiques
# ----------------------
# Les rendements logarithmiques sont privilégiés pour leur
# additivité temporelle et leur usage standard en finance empirique

returns_stocks = np.log(prices_eur / prices_eur.shift(1)).dropna()

# ----------------------
# 1.7 bis Traitement des valeurs manquantes
# ----------------------
# Les valeurs manquantes éventuelles sont remplacées par la
# médiane de chaque série afin de limiter l’impact des outliers
# et de préserver la structure de distribution des rendements

returns_stocks = returns_stocks.apply(
    lambda x: x.fillna(x.median())
)


# ----------------------
# 1.8 Construction du portefeuille GARP équipondéré
# ----------------------
# Chaque actif reçoit un poids constant identique

n_assets = returns_stocks.shape[1]
weights = np.repeat(1 / n_assets, n_assets)

# Rendement mensuel du portefeuille GARP
garp_portfolio_returns = returns_stocks.dot(weights)
garp_portfolio_returns.name = "GARP_Portfolio"


# ----------------------
# 1.9 Sauvegarde des données
# ----------------------
DATA_PROCESSED = Path("../data/processed")
DATA_PROCESSED.mkdir(exist_ok=True)

returns_stocks.to_csv(DATA_PROCESSED / "garp_stocks_returns.csv")
garp_portfolio_returns.to_csv(DATA_PROCESSED / "garp_portfolio_returns.csv")


# 2. Analyse descriptive & performance globale — Portefeuille GARP vs MSCI World

Cette section présente une analyse complète de la performance et du profil de risque du portefeuille GARP en comparaison avec le benchmark MSCI World. Nous procédons aux étapes suivantes :

- Importation et préparation des données de performance mensuelle.
- Calcul des rendements logarithmiques et alignement temporel des séries.
- Calcul et visualisation des performances cumulées et des drawdowns.
- Estimation des statistiques descriptives classiques et avancées (rendement annualisé, volatilité, Sharpe, skewness, kurtosis, Sortino, VaR, CVaR).
- Calcul de l’Information Ratio et du Tracking Error pour mesurer la valeur ajoutée du portefeuille par rapport au benchmark.
- Estimation du modèle CAPM pour extraire alpha, beta et leur significativité.
- Test statistique de la robustesse du Sharpe ratio.
- Calcul du downside beta pour évaluer le comportement en marché baissier.
- Analyse de corrélation entre GARP et MSCI World.
- Export des résultats sous forme de tableaux CSV et graphiques.

In [3]:
# ======================================================
# 2. ANALYSE DESCRIPTIVE & PERFORMANCE GLOBALE
# Portefeuille GARP vs MSCI World
# ======================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from pathlib import Path
from scipy import stats

sns.set_theme(style="whitegrid")

# -----------------------------
# 2.1 Chemins
# -----------------------------
DATA_RAW = Path("../data/raw")
RESULTS_FIGURES = Path("../results/figures")
RESULTS_TABLES = Path("../results/tables")
RESULTS_FIGURES.mkdir(parents=True, exist_ok=True)
RESULTS_TABLES.mkdir(parents=True, exist_ok=True)

# -----------------------------
# 2.2 Import MSCI World
# -----------------------------
msci_world = pd.read_excel(DATA_RAW / "msci_world_factset.xlsx")

msci_world.iloc[:, 0] = pd.to_datetime(msci_world.iloc[:, 0])
msci_world.set_index(msci_world.columns[0], inplace=True)
msci_world = msci_world.sort_index().iloc[:, 0]
msci_world.name = "MSCI_World"

# Log returns MSCI
r_msci = np.log(msci_world / msci_world.shift(1))

# -----------------------------
# 2.3 Alignement des séries
# -----------------------------
returns_comp = pd.concat(
    [garp_portfolio_returns, r_msci], axis=1
).dropna()

returns_comp.columns = ["GARP", "MSCI_World"]

# -----------------------------
# 2.4 Paramètres globaux
# -----------------------------
rf_annual = 0.02  # hypothèse taux sans risque 2%
rf_monthly = np.log(1 + rf_annual) / 12

excess_returns = returns_comp - rf_monthly

# ======================================================
# 2.5 PERFORMANCE CUMULÉE
# ======================================================

cumulative_perf = np.exp(returns_comp.cumsum())

plt.figure(figsize=(10,6))
plt.plot(cumulative_perf["GARP"], label="GARP", linewidth=2)
plt.plot(cumulative_perf["MSCI_World"], label="MSCI World", linestyle="--")
plt.title("Performance cumulée (log returns)")
plt.legend()
plt.tight_layout()
plt.savefig(RESULTS_FIGURES / "2_cumulative_performance.png")
plt.close()

# ======================================================
# 2.6 DRAWDOWNS
# ======================================================

rolling_max = cumulative_perf.cummax()
drawdowns = cumulative_perf / rolling_max - 1

plt.figure(figsize=(10,6))
plt.plot(drawdowns["GARP"], label="GARP", linewidth=2)
plt.plot(drawdowns["MSCI_World"], label="MSCI World", linestyle="--")
plt.title("Drawdowns")
plt.legend()
plt.tight_layout()
plt.savefig(RESULTS_FIGURES / "2_drawdowns.png")
plt.close()

# ======================================================
# 2.7 STATISTIQUES DESCRIPTIVES
# ======================================================

stats_table = pd.DataFrame(index=["GARP", "MSCI_World"])

stats_table["Rendement annualisé"] = returns_comp.mean() * 12
stats_table["Volatilité annualisée"] = returns_comp.std() * np.sqrt(12)
stats_table["Sharpe"] = (
    excess_returns.mean() * 12
) / (returns_comp.std() * np.sqrt(12))

stats_table["Skewness"] = returns_comp.skew()
stats_table["Kurtosis"] = returns_comp.kurtosis()
stats_table["Max Drawdown"] = drawdowns.min()

# Sortino
downside_std = returns_comp.apply(
    lambda x: np.sqrt(np.mean(np.minimum(x - rf_monthly, 0)**2)) * np.sqrt(12)
)
stats_table["Sortino"] = (
    excess_returns.mean() * 12
) / downside_std

# VaR & CVaR 95%
stats_table["VaR 95%"] = returns_comp.quantile(0.05)
stats_table["CVaR 95%"] = returns_comp.apply(
    lambda x: x[x <= x.quantile(0.05)].mean()
)

# ======================================================
# 2.8 INFORMATION RATIO & TRACKING ERROR
# ======================================================

active_returns = returns_comp["GARP"] - returns_comp["MSCI_World"]

tracking_error = active_returns.std() * np.sqrt(12)
information_ratio = (active_returns.mean() * 12) / tracking_error

stats_table.loc["GARP", "Tracking Error"] = tracking_error
stats_table.loc["GARP", "Information Ratio"] = information_ratio

# ======================================================
# 2.9 ALPHA CAPM
# ======================================================

Y = excess_returns["GARP"]
X = sm.add_constant(excess_returns["MSCI_World"])

model = sm.OLS(Y, X).fit()

alpha = model.params["const"] * 12
beta = model.params["MSCI_World"]
alpha_pval = model.pvalues["const"]

stats_table.loc["GARP", "Alpha CAPM"] = alpha
stats_table.loc["GARP", "Beta CAPM"] = beta
stats_table.loc["GARP", "p-value Alpha"] = alpha_pval
stats_table.loc["GARP", "R² CAPM"] = model.rsquared

# ======================================================
# 2.10 TEST STATISTIQUE DU SHARPE
# Approximation asymptotique
# ======================================================

T = len(returns_comp)

def sharpe_test(sr, T):
    return sr * np.sqrt(T)

sharpe_tstat = sharpe_test(stats_table.loc["GARP","Sharpe"], T)
stats_table.loc["GARP","t-stat Sharpe"] = sharpe_tstat

# ======================================================
# 2.11 DOWNSIDE BETA
# ======================================================

market_down = returns_comp["MSCI_World"] < 0
downside_beta = np.cov(
    returns_comp["GARP"][market_down],
    returns_comp["MSCI_World"][market_down]
)[0,1] / np.var(returns_comp["MSCI_World"][market_down])

stats_table.loc["GARP","Downside Beta"] = downside_beta

# ======================================================
# 2.12 CORRÉLATION
# ======================================================

corr_matrix = returns_comp.corr()

# ======================================================
# 2.13 EXPORT DES TABLEAUX
# ======================================================

stats_table.to_csv(RESULTS_TABLES / "2_performance_statistics.csv")
corr_matrix.to_csv(RESULTS_TABLES / "2_correlation_matrix.csv")

print("\n=== TABLEAU DE PERFORMANCE ===")
print(stats_table.round(4))

print("\n=== MATRICE DE CORRÉLATION ===")
print(corr_matrix.round(4))

print("\nSection 2 terminée avec succès.")


/tmp/ipykernel_9267/89327030.py:41: Pandas4Warning: Sorting by default when concatenating all DatetimeIndex is deprecated.  In the future, pandas will respect the default of `sort=False`. Specify `sort=True` or `sort=False` to silence this message. If you see this warnings when not directly calling concat, report a bug to pandas.
  returns_comp = pd.concat(



=== TABLEAU DE PERFORMANCE ===
            Rendement annualisé  Volatilité annualisée  Sharpe  Skewness  \
GARP                     0.0972                 0.2128  0.3639   -0.1339   
MSCI_World               0.0708                 0.1234  0.4131   -0.6846   

            Kurtosis  Max Drawdown  Sortino  VaR 95%  CVaR 95%  \
GARP         -0.5317       -0.3181   0.5512  -0.0937   -0.1046   
MSCI_World    0.0988       -0.2000   0.5860  -0.0651   -0.0754   

            Tracking Error  Information Ratio  Alpha CAPM  Beta CAPM  \
GARP                0.1372             0.1928      0.0077     1.3685   
MSCI_World             NaN                NaN         NaN        NaN   

            p-value Alpha  R² CAPM  t-stat Sharpe  Downside Beta  
GARP               0.9129   0.6299         2.3864         1.1426  
MSCI_World            NaN      NaN            NaN            NaN  

=== MATRICE DE CORRÉLATION ===
              GARP  MSCI_World
GARP        1.0000      0.7937
MSCI_World  0.7937      1.00

# 3. Caractérisation stylistique — Portefeuille GARP vs MSCI World Growth & Value

Cette section analyse la caractérisation stylistique du portefeuille GARP en le comparant aux indices MSCI World Growth et MSCI World Value. L’objectif est de comprendre dans quelle mesure le portefeuille s’expose aux styles de marché « Growth » (croissance) et « Value » (valeur), par le biais des étapes suivantes :

- Import et préparation des séries historiques des indices Growth et Value.
- Calcul des rendements logarithmiques mensuels et alignement temporel avec GARP.
- Calcul et visualisation des performances cumulées des trois séries.
- Estimation des statistiques descriptives clés : rendement, volatilité, ratio de Sharpe, drawdowns, corrélations.
- Calcul des metrics de suivi de la performance active : Tracking Error et Information Ratio contre Growth et Value.
- Régression stylistique multivariée : modélisation du rendement excédentaire de GARP en fonction des facteurs Growth et Value pour obtenir alpha, beta et qualité du modèle.
- Normalisation des poids stylistiques (beta) pour interprétation.
- Analyse dynamique par rolling regression sur 36 mois des expositions Growth et Value.
- Calcul des downside betas, pour évaluer la sensibilité aux marchés baissiers Growth et Value.
- Export des résultats sous forme de tableaux CSV et graphiques.

In [4]:
# ======================================================
# 3. CARACTÉRISATION STYLISTIQUE
# GARP vs MSCI World Growth & Value
# ======================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from pathlib import Path

sns.set_theme(style="whitegrid")

# ------------------------------------------------------
# 3.1 Import des indices Growth & Value
# ------------------------------------------------------

DATA_RAW = Path("../data/raw")
RESULTS_FIGURES = Path("../results/figures")
RESULTS_TABLES = Path("../results/tables")

msci_growth = pd.read_excel(DATA_RAW / "msci_world_growth_factset.xlsx")
msci_value = pd.read_excel(DATA_RAW / "msci_world_value_factset.xlsx")

def prepare_index(df, name):
    df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0])
    df.set_index(df.columns[0], inplace=True)
    df = df.sort_index().iloc[:, 0]
    df.name = name
    return df

msci_growth = prepare_index(msci_growth, "MSCI_World_Growth")
msci_value = prepare_index(msci_value, "MSCI_World_Value")

# ------------------------------------------------------
# 3.2 Log returns
# ------------------------------------------------------

r_growth = np.log(msci_growth / msci_growth.shift(1))
r_value = np.log(msci_value / msci_value.shift(1))

returns_style = pd.concat(
    [garp_portfolio_returns, r_growth, r_value], axis=1
).dropna()

returns_style.columns = ["GARP", "Growth", "Value"]

# ------------------------------------------------------
# 3.3 Paramètres globaux
# ------------------------------------------------------

rf_annual = 0.02
rf_monthly = np.log(1 + rf_annual) / 12
excess_returns_style = returns_style - rf_monthly

# ======================================================
# 3.4 PERFORMANCE CUMULÉE
# ======================================================

cumulative_style = np.exp(returns_style.cumsum())

plt.figure(figsize=(10,6))
plt.plot(cumulative_style["GARP"], label="GARP", linewidth=2)
plt.plot(cumulative_style["Growth"], label="Growth", linestyle="--")
plt.plot(cumulative_style["Value"], label="Value", linestyle=":")
plt.title("Performance cumulée – GARP vs Growth & Value")
plt.legend()
plt.tight_layout()
plt.savefig(RESULTS_FIGURES / "3_cumulative_style.png")
plt.close()

# ======================================================
# 3.5 STATISTIQUES DESCRIPTIVES
# ======================================================

style_stats = pd.DataFrame(index=["GARP","Growth","Value"])

style_stats["Rendement annualisé"] = returns_style.mean() * 12
style_stats["Volatilité annualisée"] = returns_style.std() * np.sqrt(12)
style_stats["Sharpe"] = (
    excess_returns_style.mean() * 12
) / (returns_style.std() * np.sqrt(12))

# Drawdowns
rolling_max = cumulative_style.cummax()
drawdowns = cumulative_style / rolling_max - 1
style_stats["Max Drawdown"] = drawdowns.min()

# Corrélation
corr_matrix_style = returns_style.corr()

# ======================================================
# 3.6 INFORMATION RATIO
# ======================================================

for benchmark in ["Growth", "Value"]:
    active = returns_style["GARP"] - returns_style[benchmark]
    te = active.std() * np.sqrt(12)
    ir = (active.mean() * 12) / te
    
    style_stats.loc["GARP", f"Tracking Error vs {benchmark}"] = te
    style_stats.loc["GARP", f"IR vs {benchmark}"] = ir

# ======================================================
# 3.7 RÉGRESSION STYLISTIQUE
# GARP = alpha + b1*Growth + b2*Value
# ======================================================

Y = excess_returns_style["GARP"]
X = excess_returns_style[["Growth","Value"]]
X = sm.add_constant(X)

model_style = sm.OLS(Y, X).fit()

alpha_style = model_style.params["const"] * 12
beta_growth = model_style.params["Growth"]
beta_value = model_style.params["Value"]

style_stats.loc["GARP","Alpha (Growth+Value)"] = alpha_style
style_stats.loc["GARP","Beta Growth"] = beta_growth
style_stats.loc["GARP","Beta Value"] = beta_value
style_stats.loc["GARP","p-value Alpha"] = model_style.pvalues["const"]
style_stats.loc["GARP","R² Style Model"] = model_style.rsquared

# ======================================================
# 3.8 POIDS STYLISTIQUES NORMALISÉS
# ======================================================

beta_sum = beta_growth + beta_value
style_stats.loc["GARP","Weight Growth (norm.)"] = beta_growth / beta_sum
style_stats.loc["GARP","Weight Value (norm.)"] = beta_value / beta_sum

# ======================================================
# 3.9 ROLLING STYLE EXPOSURE (36 mois)
# ======================================================

window = 36
rolling_betas_growth = []
rolling_betas_value = []
dates = []

for i in range(window, len(excess_returns_style)):
    Y_roll = excess_returns_style["GARP"].iloc[i-window:i]
    X_roll = excess_returns_style[["Growth","Value"]].iloc[i-window:i]
    X_roll = sm.add_constant(X_roll)
    
    model_roll = sm.OLS(Y_roll, X_roll).fit()
    
    rolling_betas_growth.append(model_roll.params["Growth"])
    rolling_betas_value.append(model_roll.params["Value"])
    dates.append(excess_returns_style.index[i])

rolling_df = pd.DataFrame({
    "Beta Growth": rolling_betas_growth,
    "Beta Value": rolling_betas_value
}, index=dates)

plt.figure(figsize=(10,6))
plt.plot(rolling_df["Beta Growth"], label="Rolling Beta Growth")
plt.plot(rolling_df["Beta Value"], label="Rolling Beta Value")
plt.axhline(0, color="black", linestyle="--")
plt.title("Rolling Style Exposure (36 mois)")
plt.legend()
plt.tight_layout()
plt.savefig(RESULTS_FIGURES / "3_rolling_style_exposure.png")
plt.close()

# ======================================================
# 3.10 DOWNSIDE BETAS
# ======================================================

for benchmark in ["Growth","Value"]:
    mask = returns_style[benchmark] < 0
    downside_beta = np.cov(
        returns_style["GARP"][mask],
        returns_style[benchmark][mask]
    )[0,1] / np.var(returns_style[benchmark][mask])
    
    style_stats.loc["GARP", f"Downside Beta vs {benchmark}"] = downside_beta

# ======================================================
# 3.11 EXPORT
# ======================================================

style_stats.to_csv(RESULTS_TABLES / "3_style_statistics.csv")
corr_matrix_style.to_csv(RESULTS_TABLES / "3_style_correlation_matrix.csv")

print("\n=== TABLEAU STYLISTIQUE ===")
print(style_stats.round(4))

print("\n=== MATRICE DE CORRÉLATION ===")
print(corr_matrix_style.round(4))

print("\n=== Résumé régression stylistique ===")
print(model_style.summary())


/tmp/ipykernel_9267/4123303095.py:43: Pandas4Warning: Sorting by default when concatenating all DatetimeIndex is deprecated.  In the future, pandas will respect the default of `sort=False`. Specify `sort=True` or `sort=False` to silence this message. If you see this warnings when not directly calling concat, report a bug to pandas.
  returns_style = pd.concat(



=== TABLEAU STYLISTIQUE ===
        Rendement annualisé  Volatilité annualisée  Sharpe  Max Drawdown  \
GARP                 0.0972                 0.2128  0.3639       -0.3181   
Growth               0.0895                 0.1562  0.4464       -0.2693   
Value                0.0462                 0.1201  0.2193       -0.1438   

        Tracking Error vs Growth  IR vs Growth  Tracking Error vs Value  \
GARP                      0.1159        0.0665                   0.1806   
Growth                       NaN           NaN                      NaN   
Value                        NaN           NaN                      NaN   

        IR vs Value  Alpha (Growth+Value)  Beta Growth  Beta Value  \
GARP         0.2829               -0.0024       1.1069      0.1018   
Growth          NaN                   NaN          NaN         NaN   
Value           NaN                   NaN          NaN         NaN   

        p-value Alpha  R² Style Model  Weight Growth (norm.)  \
GARP           0.968

# 4. Décomposition factorielle — CAPM, Fama-French 3F et Carhart 4F

Cette section effectue une décomposition factorielle des rendements du portefeuille GARP en utilisant les modèles financiers classiques :

- **CAPM** : Modèle du marché avec une seule factorisation.
- **Fama-French 3 facteurs (FF3)** : Ajoute les facteurs de taille (SMB) et valeur (HML) au marché.
- **Carhart 4 facteurs (4F)** : Ajoute le facteur momentum (MOM) au modèle FF3.

Les objectifs sont :

1. Importer les facteurs et préparer les données.
2. Estimer les modèles linéaires avec erreurs robustes Newey-West.
3. Résumer les résultats en termes d’alpha, R² et betas.
4. Décomposer la performance économique annuelle selon le modèle Carhart.
5. Calculer un alpha glissant sur 36 mois pour suivre l’évolution de la surperformance.
6. Tester l’autocorrélation des résidus via le test de Ljung-Box.
7. Exporter les résultats sous forme de tableaux et graphiques.



In [5]:
# ======================================================
# 4. DECOMPOSITION FACTORIELLE
# CAPM – Fama-French 3F – Carhart 4F
# ======================================================

import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from pathlib import Path
from statsmodels.stats.diagnostic import acorr_ljungbox

# ---------------------------------------------------
# 4.1 Chemins
# ---------------------------------------------------

DATA_RAW = Path("../data/raw")
RESULTS_TABLES = Path("../results/tables")
RESULTS_FIGURES = Path("../results/figures")

RESULTS_TABLES.mkdir(parents=True, exist_ok=True)
RESULTS_FIGURES.mkdir(parents=True, exist_ok=True)

# ---------------------------------------------------
# 4.2 Import facteurs Fama-French + Momentum
# ---------------------------------------------------

ff = pd.read_excel(DATA_RAW / "fama_french_3factors.xlsx")
mom = pd.read_excel(DATA_RAW / "momentum_factor.xlsx")  # facteur MOM (Carhart)

def prepare_factor(df):
    df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0])
    df.set_index(df.columns[0], inplace=True)
    return df.sort_index()

ff = prepare_factor(ff)
mom = prepare_factor(mom)

ff.columns = ["Mkt_RF", "SMB", "HML", "RF"]
mom.columns = ["MOM"]

# Fusion facteurs
factors = pd.concat([ff, mom], axis=1)

# ---------------------------------------------------
# 4.3 Alignement avec portefeuille
# ---------------------------------------------------

data = pd.concat([garp_portfolio_returns, factors], axis=1).dropna()

data["GARP_Excess"] = data["GARP_Portfolio"] - data["RF"]

Y = data["GARP_Excess"]

# ---------------------------------------------------
# 4.4 Définition des modèles
# ---------------------------------------------------

X_capm = sm.add_constant(data[["Mkt_RF"]])
X_ff3 = sm.add_constant(data[["Mkt_RF","SMB","HML"]])
X_carhart = sm.add_constant(data[["Mkt_RF","SMB","HML","MOM"]])

# ---------------------------------------------------
# 4.5 Estimation avec erreurs robustes Newey-West
# ---------------------------------------------------

model_capm = sm.OLS(Y, X_capm).fit(cov_type="HAC", cov_kwds={"maxlags":3})
model_ff3 = sm.OLS(Y, X_ff3).fit(cov_type="HAC", cov_kwds={"maxlags":3})
model_carhart = sm.OLS(Y, X_carhart).fit(cov_type="HAC", cov_kwds={"maxlags":3})

print("\n========= CAPM =========\n")
print(model_capm.summary())

print("\n========= FAMA-FRENCH 3F =========\n")
print(model_ff3.summary())

print("\n========= CARHART 4F =========\n")
print(model_carhart.summary())

# ---------------------------------------------------
# 4.6 Tableau synthèse
# ---------------------------------------------------

def extract_results(model, name):
    return pd.Series({
        "Alpha mensuel": model.params["const"],
        "Alpha annualisé": model.params["const"] * 12,
        "p-value Alpha": model.pvalues["const"],
        "R²": model.rsquared
    }, name=name)

results_summary = pd.concat([
    extract_results(model_capm, "CAPM"),
    extract_results(model_ff3, "FF3"),
    extract_results(model_carhart, "Carhart 4F")
], axis=1).T

# Ajout des betas Carhart
for factor in ["Mkt_RF","SMB","HML","MOM"]:
    results_summary[f"Beta_{factor}"] = [
        model_capm.params.get(factor, np.nan),
        model_ff3.params.get(factor, np.nan),
        model_carhart.params.get(factor, np.nan)
    ]

print("\n======= SYNTHESE FACTORIELLE =======\n")
print(results_summary.round(4))

results_summary.to_csv(RESULTS_TABLES / "4_factor_models_summary.csv")

# ---------------------------------------------------
# 4.7 Décomposition économique (Carhart)
# ---------------------------------------------------

mean_factors = data[["Mkt_RF","SMB","HML","MOM"]].mean()
params = model_carhart.params

contribution = pd.Series({
    "Alpha": params["const"] * 12,
    "Marché": params["Mkt_RF"] * mean_factors["Mkt_RF"] * 12,
    "Taille (SMB)": params["SMB"] * mean_factors["SMB"] * 12,
    "Valeur (HML)": params["HML"] * mean_factors["HML"] * 12,
    "Momentum (MOM)": params["MOM"] * mean_factors["MOM"] * 12
})

contribution.to_csv(RESULTS_TABLES / "4_factor_contribution_carhart.csv")

plt.figure(figsize=(8,5))
contribution.plot(kind="bar")
plt.title("Décomposition de la performance – Modèle Carhart 4F")
plt.ylabel("Contribution annualisée")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(RESULTS_FIGURES / "4_factor_contribution_carhart.png")
plt.close()

# ---------------------------------------------------
# 4.8 Rolling Alpha (36 mois)
# ---------------------------------------------------

window = 36
rolling_alpha = []
dates = []

for i in range(window, len(data)):
    y_roll = data["GARP_Excess"].iloc[i-window:i]
    X_roll = sm.add_constant(data[["Mkt_RF","SMB","HML","MOM"]].iloc[i-window:i])
    
    model_roll = sm.OLS(y_roll, X_roll).fit()
    rolling_alpha.append(model_roll.params["const"] * 12)
    dates.append(data.index[i])

rolling_alpha = pd.Series(rolling_alpha, index=dates)

plt.figure(figsize=(10,6))
plt.plot(rolling_alpha, label="Rolling Alpha (Carhart)")
plt.axhline(0, color="black", linestyle="--")
plt.title("Alpha glissant (36 mois) – Carhart")
plt.legend()
plt.tight_layout()
plt.savefig(RESULTS_FIGURES / "4_rolling_alpha_carhart.png")
plt.close()

# ---------------------------------------------------
# 4.9 Test d'autocorrélation des résidus
# ---------------------------------------------------

ljung_box = acorr_ljungbox(model_carhart.resid, lags=[6], return_df=True)
ljung_box.to_csv(RESULTS_TABLES / "4_ljung_box_test.csv")

print("\n======= CONTRIBUTION ANNUALISEE =======\n")
print(contribution.round(4))

print("\n======= TEST LJUNG-BOX (résidus Carhart) =======\n")
print(ljung_box)

print("\nSection 4 terminée avec succès.")


/tmp/ipykernel_9267/2873860974.py:49: Pandas4Warning: Sorting by default when concatenating all DatetimeIndex is deprecated.  In the future, pandas will respect the default of `sort=False`. Specify `sort=True` or `sort=False` to silence this message. If you see this warnings when not directly calling concat, report a bug to pandas.
  data = pd.concat([garp_portfolio_returns, factors], axis=1).dropna()



========= CAPM =========

                            OLS Regression Results                            
Dep. Variable:            GARP_Excess   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     2.073
Date:                Wed, 11 Feb 2026   Prob (F-statistic):              0.155
Time:                        22:17:48   Log-Likelihood:                 18.393
No. Observations:                  58   AIC:                            -32.79
Df Residuals:                      56   BIC:                            -28.67
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2610    

# 5. Persistance du couple rendement–risque

Cette section analyse la **stabilité et la persistance** des performances du portefeuille GARP dans le temps, à travers plusieurs indicateurs et tests statistiques :

- **Rolling Sharpe ratio (12 mois)** : pour observer la variation du rendement ajusté du risque.
- **Rolling Information Ratio vs MSCI World (12 mois)** : pour suivre la surperformance relative.
- **Rolling Alpha (Carhart 4F, 36 mois)** : pour examiner la persistance du rendement excédentaire ajusté aux facteurs.
- **Test AR(1) sur les rendements** : pour détecter l’auto-corrélation.
- **Auto-corrélation du Sharpe ratio** : pour analyser la persistance du couple rendement/risque.
- **Analyse par régimes de marché (bull / bear)** : pour comparer la performance selon le contexte.
- **Test de stabilité structurelle (CUSUM)** : pour identifier d’éventuels changements dans la dynamique du portefeuille.

In [6]:
# ======================================================
# 5. PERSISTANCE DU COUPLE RENDEMENT–RISQUE
# ======================================================

import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from pathlib import Path
from statsmodels.stats.diagnostic import breaks_cusumolsresid
from scipy import stats

RESULTS_TABLES = Path("../results/tables")
RESULTS_FIGURES = Path("../results/figures")

RESULTS_TABLES.mkdir(parents=True, exist_ok=True)
RESULTS_FIGURES.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------
# 5.1 Paramètres
# ------------------------------------------------------

rf_annual = 0.02
rf_monthly = np.log(1 + rf_annual) / 12

window = 12

excess_returns = garp_portfolio_returns - rf_monthly

# ======================================================
# 5.2 Rolling Sharpe (12 mois)
# ======================================================

rolling_mean = excess_returns.rolling(window).mean() * 12
rolling_vol = garp_portfolio_returns.rolling(window).std() * np.sqrt(12)

rolling_sharpe = rolling_mean / rolling_vol

plt.figure(figsize=(10,6))
plt.plot(rolling_sharpe, label="Rolling Sharpe (12m)")
plt.axhline(0, color="black", linestyle="--")
plt.title("Rolling Sharpe Ratio – GARP")
plt.legend()
plt.tight_layout()
plt.savefig(RESULTS_FIGURES / "5_rolling_sharpe.png")
plt.close()

rolling_sharpe.to_csv(RESULTS_TABLES / "5_rolling_sharpe.csv")

# ======================================================
# 5.3 Rolling Information Ratio vs MSCI World
# ======================================================

active_returns = returns_comp["GARP"] - returns_comp["MSCI_World"]

rolling_active_mean = active_returns.rolling(window).mean() * 12
rolling_tracking_error = active_returns.rolling(window).std() * np.sqrt(12)

rolling_ir = rolling_active_mean / rolling_tracking_error

plt.figure(figsize=(10,6))
plt.plot(rolling_ir, label="Rolling Information Ratio")
plt.axhline(0, color="black", linestyle="--")
plt.title("Rolling Information Ratio – GARP vs MSCI World")
plt.legend()
plt.tight_layout()
plt.savefig(RESULTS_FIGURES / "5_rolling_information_ratio.png")
plt.close()

rolling_ir.to_csv(RESULTS_TABLES / "5_rolling_information_ratio.csv")

# ======================================================
# 5.4 Rolling Alpha (Carhart 4F)
# ======================================================

window_reg = 36
rolling_alpha = []
dates = []

for i in range(window_reg, len(data)):
    
    sub = data.iloc[i-window_reg:i]
    y_sub = sub["GARP_Excess"]
    X_sub = sm.add_constant(sub[["Mkt_RF","SMB","HML","MOM"]])
    
    model_sub = sm.OLS(y_sub, X_sub).fit()
    rolling_alpha.append(model_sub.params["const"] * 12)
    dates.append(data.index[i])

rolling_alpha = pd.Series(rolling_alpha, index=dates)

plt.figure(figsize=(10,6))
plt.plot(rolling_alpha, label="Rolling Alpha (Carhart)")
plt.axhline(0, color="black", linestyle="--")
plt.title("Rolling Alpha (36 mois)")
plt.legend()
plt.tight_layout()
plt.savefig(RESULTS_FIGURES / "5_rolling_alpha_carhart.png")
plt.close()

rolling_alpha.to_csv(RESULTS_TABLES / "5_rolling_alpha_carhart.csv")

# ======================================================
# 5.5 Test de persistance AR(1)
# ======================================================

ar_model = sm.OLS(
    garp_portfolio_returns[1:],
    sm.add_constant(garp_portfolio_returns.shift(1)[1:])
).fit()

phi = ar_model.params.iloc[1]  # accès par position
p_value_phi = ar_model.pvalues.iloc[1]

ar_results = pd.Series({
    "Coefficient AR(1)": phi,
    "p-value": p_value_phi,
    "R²": ar_model.rsquared
})

ar_results.to_csv(RESULTS_TABLES / "5_ar1_persistence_test.csv")

# ======================================================
# 5.6 Autocorrélation du Sharpe
# ======================================================

sharpe_series = rolling_sharpe.dropna()

ar_sharpe = sm.OLS(
    sharpe_series[1:],
    sm.add_constant(sharpe_series.shift(1)[1:])
).fit()

sharpe_persistence = pd.Series({
    "AR(1) Sharpe": ar_sharpe.params.iloc[1],
    "p-value": ar_sharpe.pvalues.iloc[1]
})

sharpe_persistence.to_csv(RESULTS_TABLES / "5_sharpe_persistence_test.csv")

# ======================================================
# 5.7 Analyse par régimes de marché
# ======================================================

# Alignement temporel des séries
aligned_data = pd.concat([
    garp_portfolio_returns,
    returns_comp["MSCI_World"]
], axis=1).dropna()

aligned_data.columns = ["GARP", "MSCI_World"]

# Créer les masques à partir des données alignées
bull = aligned_data["MSCI_World"] > 0
bear = aligned_data["MSCI_World"] <= 0

def regime_stats(mask):
    series = aligned_data["GARP"][mask]
    return pd.Series({
        "Rendement annualisé": series.mean() * 12,
        "Volatilité annualisée": series.std() * np.sqrt(12),
        "Sharpe": (series.mean() * 12) / (series.std() * np.sqrt(12))
    })

regime_analysis = pd.DataFrame({
    "Bull Market": regime_stats(bull),
    "Bear Market": regime_stats(bear)
})

regime_analysis.to_csv(RESULTS_TABLES / "5_regime_analysis.csv")

# ======================================================
# 5.8 Test de stabilité structurelle (CUSUM)
# ======================================================

cusum_test = breaks_cusumolsresid(ar_model.resid, ddof=1)

cusum_results = pd.Series({
    "CUSUM Statistic": cusum_test[0],
    "p-value": cusum_test[1]
})

cusum_results.to_csv(RESULTS_TABLES / "5_cusum_stability_test.csv")

# ======================================================
# 5.9 Affichage synthétique
# ======================================================

print("\n===== TEST AR(1) RENDEMENTS =====")
print(ar_results)

print("\n===== PERSISTANCE SHARPE =====")
print(sharpe_persistence)

print("\n===== ANALYSE REGIMES =====")
print(regime_analysis)

print("\n===== TEST CUSUM =====")
print(cusum_results)

print("\nSection 5 terminée avec succès.")



===== TEST AR(1) RENDEMENTS =====
Coefficient AR(1)   -0.115534
p-value              0.388275
R²                   0.013321
dtype: float64

===== PERSISTANCE SHARPE =====
AR(1) Sharpe    9.066860e-01
p-value         2.612537e-17
dtype: float64

===== ANALYSE REGIMES =====
                       Bull Market  Bear Market
Rendement annualisé       0.512984    -0.538591
Volatilité annualisée     0.153328     0.151431
Sharpe                    3.345669    -3.556683

===== TEST CUSUM =====
CUSUM Statistic    0.749594
p-value            0.627851
dtype: float64

Section 5 terminée avec succès.


# 6. Analyse de robustesse

Cette section évalue la **robustesse des résultats** du portefeuille GARP face à différents choix méthodologiques, périodes et coûts. Les tests effectués incluent :

- **Benchmark alternatif** : comparaison avec le FTSE All-World.
- **Bootstrap de l’alpha** : estimation de la distribution de l’alpha pour vérifier sa significativité.
- **Sous-périodes** : analyse des performances sur différentes périodes.
- **Impact des coûts de transaction** : ajustement des rendements pour des frais hypothétiques.
- **Spécification des rendements** : log returns vs simple returns.
- **Exclusion d’une période extrême** :  tester la sensibilité aux événements exceptionnels (exemple : 2022).



In [7]:
# ======================================================
# 6. ANALYSE DE ROBUSTESSE
# ======================================================

import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from pathlib import Path

RESULTS_FIGURES = Path("../results/figures")
RESULTS_TABLES = Path("../results/tables")

RESULTS_FIGURES.mkdir(parents=True, exist_ok=True)
RESULTS_TABLES.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------
# 6.1 ROBUSTESSE AU BENCHMARK – FTSE ALL-WORLD
# ------------------------------------------------------

ftse_aw = pd.read_excel(Path("../data/raw/ftse_all_world_factset.xlsx"))

ftse_aw.iloc[:,0] = pd.to_datetime(ftse_aw.iloc[:,0])
ftse_aw.set_index(ftse_aw.columns[0], inplace=True)
ftse_aw = ftse_aw.sort_index().iloc[:,0]
ftse_aw.name = "FTSE_All_World"

r_ftse = np.log(ftse_aw / ftse_aw.shift(1))

returns_robust = pd.concat(
    [garp_portfolio_returns, r_ftse], axis=1
).dropna()

returns_robust.columns = ["GARP","FTSE"]

# Performance cumulée (log cohérent)
cumulative = np.exp(returns_robust.cumsum())

plt.figure(figsize=(10,6))
plt.plot(cumulative["GARP"], label="GARP")
plt.plot(cumulative["FTSE"], linestyle="--", label="FTSE All-World")
plt.title("Robustesse – Benchmark alternatif")
plt.legend()
plt.tight_layout()
plt.savefig(RESULTS_FIGURES / "6_benchmark_robustesse.png")
plt.close()

# Information Ratio vs FTSE
active = returns_robust["GARP"] - returns_robust["FTSE"]
tracking_error = active.std() * np.sqrt(12)
information_ratio = (active.mean() * 12) / tracking_error

benchmark_stats = pd.Series({
    "Tracking Error": tracking_error,
    "Information Ratio": information_ratio
})

benchmark_stats.to_csv(RESULTS_TABLES / "6_ftse_information_ratio.csv")

# ------------------------------------------------------
# 6.2 ROBUSTESSE STATISTIQUE – BOOTSTRAP ALPHA
# ------------------------------------------------------

n_boot = 5000
alpha_boot = []

rf_annual = 0.02
rf_monthly = np.log(1 + rf_annual) / 12

excess = returns_robust["GARP"] - rf_monthly
market_excess = returns_robust["FTSE"] - rf_monthly

for _ in range(n_boot):
    
    sample_idx = np.random.choice(len(excess), len(excess), replace=True)
    
    y = excess.iloc[sample_idx]
    X = sm.add_constant(market_excess.iloc[sample_idx])
    
    model = sm.OLS(y, X).fit()
    alpha_boot.append(model.params["const"] * 12)

alpha_boot = np.array(alpha_boot)

ci_lower = np.percentile(alpha_boot, 2.5)
ci_upper = np.percentile(alpha_boot, 97.5)

bootstrap_results = pd.Series({
    "Alpha moyen (bootstrap)": alpha_boot.mean(),
    "IC 2.5%": ci_lower,
    "IC 97.5%": ci_upper
})

bootstrap_results.to_csv(RESULTS_TABLES / "6_bootstrap_alpha.csv")

plt.figure(figsize=(8,5))
plt.hist(alpha_boot, bins=40)
plt.axvline(ci_lower, color="red", linestyle="--")
plt.axvline(ci_upper, color="red", linestyle="--")
plt.title("Distribution Bootstrap de l'Alpha (annualisé)")
plt.tight_layout()
plt.savefig(RESULTS_FIGURES / "6_bootstrap_alpha_distribution.png")
plt.close()

# ------------------------------------------------------
# 6.3 ROBUSTESSE AUX SOUS-PÉRIODES
# ------------------------------------------------------

mid = len(returns_robust)//2
sub1 = returns_robust.iloc[:mid]
sub2 = returns_robust.iloc[mid:]

def perf(df):
    return pd.Series({
        "Rendement annualisé": df["GARP"].mean()*12,
        "Volatilité annualisée": df["GARP"].std()*np.sqrt(12),
        "Sharpe": (df["GARP"].mean()*12)/(df["GARP"].std()*np.sqrt(12))
    })

subperiod_results = pd.DataFrame({
    "Sous-période 1": perf(sub1),
    "Sous-période 2": perf(sub2)
})

subperiod_results.to_csv(RESULTS_TABLES / "6_subperiod_robustesse.csv")

# ------------------------------------------------------
# 6.4 ROBUSTESSE AUX COÛTS DE TRANSACTION
# ------------------------------------------------------

transaction_cost = 0.002  # 20 bps par mois hypothétique

returns_net = garp_portfolio_returns - transaction_cost/12

net_stats = pd.Series({
    "Rendement annualisé net": returns_net.mean()*12,
    "Sharpe net": (returns_net.mean()*12)/(returns_net.std()*np.sqrt(12))
})

net_stats.to_csv(RESULTS_TABLES / "6_transaction_cost_impact.csv")

# ------------------------------------------------------
# 6.5 ROBUSTESSE AUX SPECIFICATIONS
# Log returns vs Simple returns
# ------------------------------------------------------

simple_returns = np.exp(garp_portfolio_returns) - 1

spec_comparison = pd.Series({
    "Rendement annualisé (log)": garp_portfolio_returns.mean()*12,
    "Rendement annualisé (simple)": simple_returns.mean()*12
})

spec_comparison.to_csv(RESULTS_TABLES / "6_specification_comparison.csv")

# ------------------------------------------------------
# 6.6 EXCLUSION D'UNE PERIODE EXTREME
# ------------------------------------------------------

# Exemple : exclusion année 2022
filtered = garp_portfolio_returns[garp_portfolio_returns.index.year != 2022]

extreme_test = pd.Series({
    "Rendement annualisé (complet)": garp_portfolio_returns.mean()*12,
    "Rendement annualisé (sans 2022)": filtered.mean()*12
})

extreme_test.to_csv(RESULTS_TABLES / "6_extreme_period_test.csv")

# ------------------------------------------------------
# 6.7 AFFICHAGE SYNTHÉTIQUE
# ------------------------------------------------------

print("\n===== ROBUSTESSE BENCHMARK =====")
print(benchmark_stats)

print("\n===== BOOTSTRAP ALPHA =====")
print(bootstrap_results)

print("\n===== IMPACT COUTS DE TRANSACTION =====")
print(net_stats)

print("\nSection 6 terminée avec succès.")


/tmp/ipykernel_9267/695909280.py:30: Pandas4Warning: Sorting by default when concatenating all DatetimeIndex is deprecated.  In the future, pandas will respect the default of `sort=False`. Specify `sort=True` or `sort=False` to silence this message. If you see this warnings when not directly calling concat, report a bug to pandas.
  returns_robust = pd.concat(



===== ROBUSTESSE BENCHMARK =====
Tracking Error       0.141496
Information Ratio    0.236795
dtype: float64

===== BOOTSTRAP ALPHA =====
Alpha moyen (bootstrap)    0.011301
IC 2.5%                   -0.126268
IC 97.5%                   0.145766
dtype: float64

===== IMPACT COUTS DE TRANSACTION =====
Rendement annualisé net    0.137770
Sharpe net                 0.660482
dtype: float64

Section 6 terminée avec succès.
